<a href="https://colab.research.google.com/github/MariaTSP/Projeto-SQLite-GoogleColab/blob/main/Projeto_BD_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
#Projeto de um sistema de doações.
#Conectando ao banco de dados e ativando foreign keys

import sqlite3

# Criando a conexão com o banco de dados
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Ativar chaves estrangeiras
cursor.execute("PRAGMA foreign_keys = ON;")

#-------------------------------------------------------------
#Criação das tabelas

#Criação da tabela usuarios

cursor.execute('''
  CREATE TABLE IF NOT EXISTS usuarios(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    tipo TEXT CHECK (tipo IN('doador','beneficiario'))
    ); ''')

#Criação da tabela categorias

cursor.execute('''
  CREATE TABLE IF NOT EXISTS categorias(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL
  );
''')

#Criação da tabela itens

cursor.execute('''
  CREATE TABLE IF NOT EXISTS itens(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    descricao TEXT,
    quantidade INTEGER,
    id_categoria INTEGER,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id)
  );
''')

#Criação da tabela solicitacoes

cursor.execute('''
  CREATE TABLE IF NOT EXISTS solicitacoes(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_usuario INTEGER NOT NULL,
    id_item INTEGER NOT NULL,
    data DATETIME DEFAULT (DATETIME('now')),
    FOREIGN KEY (id_usuario) REFERENCES usuarios(id),
    FOREIGN KEY (id_item) REFERENCES itens(id)
  );
''')

conn.commit()


In [105]:
#----------------------------------------------------------
#Inserindo dados na tabela usuarios

cursor.execute('''
  INSERT INTO usuarios (nome, email, tipo) VALUES
  ('Maria das Dores', 'maria@gmail.com', 'beneficiario'),
  ('João', 'joaozinho@gmail.com', 'beneficiario'),
  ('Igreja batista', 'igrejabatista@gmail.com', 'doador');
''')
conn.commit()

In [106]:
#------------------------------------------------------------
#Select dos dados da tabela usuarios

conn.commit()
cursor.execute("SELECT * FROM usuarios;")
results = cursor.fetchall()
for row in results:
  print (row)


(1, 'Maria das Dores', 'maria@gmail.com', 'beneficiario')
(2, 'João', 'joaozinho@gmail.com', 'beneficiario')
(3, 'Igreja batista', 'igrejabatista@gmail.com', 'doador')


In [107]:
#--------------------------------------------
#Inserindo dados na tabela categorias

cursor.execute('''
  INSERT INTO categorias (nome) VALUES
  ('Alimentos'),
  ('Roupas'),
  ('Higiene');
''')
conn.commit()

In [108]:
#----------------------------------------------
#Select dos dados da tabela categorias

cursor.execute("SELECT * FROM categorias;")
results = cursor.fetchall()

for row in results:
  print(row)

(1, 'Alimentos')
(2, 'Roupas')
(3, 'Higiene')


In [109]:
#------------------------------------------------------------
#Inserindo dados na tabela itens

cursor.execute('''
  INSERT INTO itens (nome, descricao, quantidade, id_categoria) VALUES
  ('Cesta Básica', 'Contém arroz, feijão, farinha e sardinha',7, 1),
  ('Roupa infantil', 'Vestido tamanho 4, usado poucas vezes.',1, 2),
  ('Máscaras', 'pacote com 10 unidades de mascara descartável',15, 3);
''')
conn.commit()

In [110]:
#------------------------------------------------------------------
#select dos dados da tabela itens

cursor.execute("SELECT * FROM itens;")
results = cursor.fetchall()

for row in results:
    print(row)

(1, 'Cesta Básica', 'Contém arroz, feijão, farinha e sardinha', 7, 1)
(2, 'Roupa infantil', 'Vestido tamanho 4, usado poucas vezes.', 1, 2)
(3, 'Máscaras', 'pacote com 10 unidades de mascara descartável', 15, 3)


In [135]:
#------------------------------------------------------------
#Inserindo dados na tabela solicitacoes

cursor.execute('''
  INSERT INTO solicitacoes (id_usuario, id_item) VALUES
  (1, 2),
  (2, 3),
  (3, 1),
  (2, 1),
  (1, 1),
  (1, 3);
''')
conn.commit()

In [136]:
#--------------------------------------------------------------
#Select dos dados da tabela solicitacoes

cursor.execute("SELECT * FROM solicitacoes")
results = cursor.fetchall()

for row in results:
  print(row)

(1, 1, 2, '2025-04-11 02:21:13')
(2, 2, 3, '2025-04-11 02:21:13')
(3, 3, 1, '2025-04-11 02:21:13')
(4, 2, 1, '2025-04-11 02:21:13')
(5, 1, 1, '2025-04-11 02:21:13')
(6, 1, 3, '2025-04-11 02:21:13')


In [113]:
#-------------------------------------------------------------
#select utilizando join para listar todos os itens e suas categorias

cursor.execute('''
  SELECT itens.nome AS item, categorias.nome AS categoria
  FROM itens
  JOIN categorias ON itens.id_categoria = categorias.id;
''')
for row in cursor.fetchall():
  print(row)

('Cesta Básica', 'Alimentos')
('Roupa infantil', 'Roupas')
('Máscaras', 'Higiene')


In [114]:
#-----------------------------------------------------------
#select utilizando join
#mostra quem e quando solicitou o item

cursor.execute('''
  SELECT usuarios.nome AS solicitante, itens.nome AS item, solicitacoes.data
  FROM solicitacoes
  JOIN usuarios ON solicitacoes.id_usuario = usuarios.id
  JOIN itens ON solicitacoes.id_item = itens.id;
''')
for row in cursor.fetchall():
  print(row)

('Maria das Dores', 'Roupa infantil', '2025-04-11 02:15:53')
('João', 'Máscaras', '2025-04-11 02:15:53')
('Igreja batista', 'Cesta Básica', '2025-04-11 02:15:53')


In [139]:
#----------------------------------------------
#select utilizando group by
#faz um ranking de itens mais solicitados

cursor.execute('''
  SELECT item.nome, COUNT(s.id) AS total_solicitacoes
  FROM itens item
  LEFT JOIN solicitacoes s ON s.id_item = item.id
  GROUP BY item.nome
  ORDER BY total_solicitacoes DESC;
''')
for row in cursor.fetchall():
  print(row)

('Cesta Básica', 3)
('Máscaras', 2)
('Roupa infantil', 1)
